# Phase 4 – Generative AI Integration (CIMA Dataset)

## Objective

In this phase, we integrate a Generative AI model to work with the CIMA films dataset.

**Goal:**  
The user writes a description of a film in Arabic, and the AI predicts the most likely **film genre** (تصنيف الفيلم) based on patterns it has learned from data and general knowledge.

We will:

1. Load the CIMA dataset.
2. Extract the list of possible genres from the dataset.
3. Design **two different prompt templates** for genre prediction:
   - Template 1: Short, direct classification.
   - Template 2: Detailed explanation with reasoning.
4. Connect to a Generative AI model (e.g., GPT) via API.
5. Test both templates using:
   - User-like descriptions we create.
   - Real film summaries (ملخص) from the CIMA dataset.
6. Compare the outputs and justify which template is better for the final system.


In [122]:
import os
import pandas as pd
!pip install groq
!pip install -q groq






In [123]:
import os
os.environ["GROQ_API_KEY"] = "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"


from groq import Groq

client = Groq(api_key="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")




In [124]:
# Load cleaned CIMA dataset (adjust path if needed)
df = pd.read_excel("cleaned_cima_dataset.xlsx")

# Show first rows
df.head()


,اسم الفيلم,تاريخ العرض,تصنيف الفيلم,مدة الفيلم (دقيقة),ملخص,تأليف,تمثيل,إنتاج,تصوير,مونتاج,ديكور,ملابس,موسيقى,إخراج,توزيع
0,قلب المرأة,1940,رومانسي,0.062944,تدور أحداث الفيلم حول (خيرية)، الفتاة الثرية ا...,توجو مزراحي (أحمد المشرقي),"سليمان نجيب,أمينة رزق,دولت أبيض,عقيلة راتب,سلو...",الأفلام المتحدة (أنور وجدي وشركاه),عبدالحليم نصر,جلال مصطفى,عباس حلمي,أحمد الكسار,فؤاد الظاهري,توجو مزراحي (أحمد المشرقي),منتخبات بهنا فيلم
1,الورشة,1940,دراما,0.081218,يسافر الأسطى (علي عبدالرحمن) صاحب ورشة الميكان...,"عزيزة أمير,محمود ذو الفقار","عزيزة أمير,محمود ذو الفقار,أنور وجدي,نجمة إبرا...",إيزيس فيلم,"أرام ماراليان,فيرى فاركاش (فرانسوا فاركاش)",جلال مصطفى,محمد كامل,أحمد الكسار,عبدالحميد عبدالرحمن,إستيفان روستي,منتخبات بهنا فيلم
2,تحت السلاح,1940,دراما,0.082185,أمينة فتاة من أسرة عريقة، تقرر العائلة زواجها ...,أبو السعود الإبياري,"أحمد علام,زوزو شكيب,عباس فارس,محمد الديب,زينات...",ألفيزى اورفانيللى,عبدالحليم نصر,جلال مصطفى,عباس حلمي,أحمد الكسار,فؤاد الظاهري,فؤاد الجزايرلي,منتخبات بهنا فيلم
3,حياة الظلام,1940,دراما,0.070051,عائلة متوسطة الحال مكونة من زوج وزوجة وابنهما ...,"محمود كامل حسن,أحمد بدرخان","ميمي شكيب,محسن سرحان,روحية خالد,فردوس محمد,علي...",محمد جمال الدين رفعت,محمد عبدالعظيم,جمال مدكور,"علي عابد,روبرت شارفنبرج",أحمد الكسار,"بيرم التونسي,عبدالحميد عبدالرحمن,محمد الكحلاوي...","أحمد بدرخان,فريد الجندي",منتخبات بهنا فيلم
4,صرخة في الليل,1940,دراما,0.086294,لا تحفظ الزوجة الساقطة شرف زوجها الثري ترتبط ب...,السيد زيادة,"بدر لاما,رجاء عبده,سميحة سميح,بشارة واكيم,منسى...",كوندور فيلم,إبراهيم شيبا,"بدر لاما,ألبير نجيب",محروس زيادة,أحمد الكسار,"فريد غصن,يوسف صالح",إبراهيم لاما,منتخبات بهنا فيلم


In [125]:
print("Columns in the dataset:\n", df.columns.tolist())

# Extract unique genres from تصنيف الفيلم
genres = sorted(df["تصنيف الفيلم"].dropna().unique().tolist())
print("\nNumber of unique genres:", len(genres))
print("\nGenres list:")
for g in genres:
    print("-", g)


Columns in the dataset:
 ['اسم الفيلم', 'تاريخ العرض', 'تصنيف الفيلم', 'مدة الفيلم (دقيقة)', 'ملخص', 'تأليف', 'تمثيل', 'إنتاج', 'تصوير', 'مونتاج', 'ديكور', 'ملابس', 'موسيقى', 'إخراج', 'توزيع']

Number of unique genres: 20

Genres list:
- استعراضي
- تاريخي
- جريمة
- حرب
- حركة
- خيال
- خيال علمي
- دراما
- رعب
- رومانسي
- رياضي
- سيرة ذاتية
- عائلي
- غموض
- قصير
- كوميدي
- مغامرات
- وثائقي
- ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ
- ﺭﺳﻮﻡ ﻣﺘحركة


## Dataset Overview

The CIMA dataset contains information about Arabic films, including:

- **اسم الفيلم** – Film name  
- **تاريخ العرض** – Release date  
- **تصنيف الفيلم** – Film genre (target label for our AI)  
- **مدة الفيلم (دقيقة)** – Duration in minutes  
- **ملخص** – Plot summary  
- Crew and production fields such as: تأليف، تمثيل، إنتاج، تصوير، مونتاج، ديكور، ملابس، موسيقى، إخراج، توزيع.

In this phase, we mainly use:

- **ملخص** (plot description) as input text.  
- **تصنيف الفيلم** as the ground truth genre.


In [126]:
genres_list_str = ", ".join(genres)
genres_list_str


'استعراضي, تاريخي, جريمة, حرب, حركة, خيال, خيال علمي, دراما, رعب, رومانسي, رياضي, سيرة ذاتية, عائلي, غموض, قصير, كوميدي, مغامرات, وثائقي, ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ, ﺭﺳﻮﻡ ﻣﺘحركة'

## Prompt Design

We designed **two different prompt templates** to predict the film genre from a text description:

1. **Template 1 – Direct Genre Classification**  
   - Short and strict.
   - Asks the model to return *only* the genre (one of the CIMA genres).

2. **Template 2 – Analytical Genre Prediction**  
   - Asks for:
     - Main genre
     - Explanation / reasoning
     - Optional secondary genre
   - Useful for debugging and understanding the model’s behaviour.

We will compare both templates and then select one for the final system.

In [127]:
# Template 1: Short, direct classification
TEMPLATE_1 = f"""
أنت مساعد متخصص في تصنيف الأفلام.

سيقوم المستخدم بكتابة وصف / ملخص لفيلم باللغة العربية.
مهمتك هي توقع نوع الفيلم (تصنيف الفيلم) بالاعتماد على الوصف فقط.

### ملاحظات مهمة:
- اختر النوع من القائمة التالية فقط:
  {genres_list_str}
- لا تكتب أي شيء آخر غير نوع الفيلم نفسه.
- لا تشرح، فقط اكتب التصنيف النهائي.

وصف المستخدم:
"{{description}}"

نوع الفيلم المتوقع:
"""

# Template 2: Detailed explanation with reasoning
TEMPLATE_2 = f"""
أنت ناقد سينمائي وخبير في تصنيف الأفلام.

سيتم إعطاؤك وصفًا/ملخصًا لفيلم باللغة العربية.
مهمتك:

1. تحديد نوع الفيلم (التصنيف الأساسي) من القائمة التالية فقط:
   {genres_list_str}
2. شرح سبب اختيارك لهذا النوع بالاعتماد على أحداث الوصف.
3. إذا كان هناك غموض، اقترح تصنيفًا ثانويًا محتملًا مع سبب مختصر.

اكتب إجابتك بهذا الشكل:

- النوع الرئيسي:
- التبرير:
- تصنيف ثانوي محتمل (إن وجد):

وصف الفيلم:
"{{description}}"
"""


In [128]:


def generate_genre_prediction(description, template, model="llama-3.1-8b-instant"):
    try:
        prompt = template.format(description=description)

        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )

        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"[Error: {str(e)}]"





In [129]:
test_description = """
تدور أحداث الفيلم حول شاب فقير يقع في حب فتاة غنية،
لكن عائلتها ترفض هذه العلاقة بسبب الفوارق الاجتماعية،
ويحاولان التمسك ببعضهما رغم كل الصعوبات.
"""

print("=== TEMPLATE 1 OUTPUT ===")
out1 = generate_genre_prediction(test_description, TEMPLATE_1)
print(out1)

print("\n=== TEMPLATE 2 OUTPUT ===")
out2 = generate_genre_prediction(test_description, TEMPLATE_2)
print(out2)


=== TEMPLATE 1 OUTPUT ===
رومانسي

=== TEMPLATE 2 OUTPUT ===
- النوع الرئيسي: رومانسي
- التبرير: الفيلم يركز على قصة حب بين شخصين من طبقات اجتماعية مختلفة، ويحاولان التمسك ببعضهما رغم الصعوبات التي تواجههم، مما يجعله فيلمًا رومانسيًا.
- تصنيف ثانوي محتمل: دراما
(بسبب التأكيد على الصعوبات التي تواجه الشاب والفتاة في علاقتهما، وبالتالي يحتوي الفيلم على عناصر درامية).


In [130]:
# Select a small sample of films from the dataset
sample_df = df[["اسم الفيلم", "تصنيف الفيلم", "ملخص"]].dropna().sample(5, random_state=42)

results = []

for _, row in sample_df.iterrows():
    name = row["اسم الفيلم"]
    true_genre = row["تصنيف الفيلم"]
    summary = row["ملخص"]

    pred_1 = generate_genre_prediction(summary, TEMPLATE_1)
    pred_2 = generate_genre_prediction(summary, TEMPLATE_2)

    results.append({
        "اسم الفيلم": name,
        "التصنيف الحقيقي": true_genre,
        "توقع Template 1": pred_1,
        "توقع Template 2": pred_2
    })

results_df = pd.DataFrame(results)
results_df


,اسم الفيلم,التصنيف الحقيقي,توقع Template 1,توقع Template 2
0,ملك البترول,كوميدي,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يركز...
1,الكل يغنى,دراما,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يركز...
2,وغدا,دراما,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يركز...
3,نداء الحب,دراما,رومانسي,- النوع الرئيسي: رومانسي\n- التبرير: الفيلم ير...
4,كابتن مصر,رياضي,رومانسي,- النوع الرئيسي: رياضي\n- التبرير: يُشير الوصف...


In [131]:
def generate_genre_prediction(description, template, model="llama-3.1-8b-instant"):
    try:
        prompt = template.format(description=description)

        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"[Error: {str(e)}]"




In [132]:
# Select a random sample of 10 films from your CIMA dataset
sample_df = df[["اسم الفيلم", "تصنيف الفيلم", "ملخص"]].dropna().sample(10, random_state=42)

results = []

for _, row in sample_df.iterrows():
    name = row["اسم الفيلم"]
    true_genre = row["تصنيف الفيلم"]
    summary = row["ملخص"]

    pred_1 = generate_genre_prediction(summary, TEMPLATE_1)
    pred_2 = generate_genre_prediction(summary, TEMPLATE_2)

    results.append({
        "Film Name": name,
        "True Genre": true_genre,
        "Template 1 Prediction": pred_1,
        "Template 2 Prediction": pred_2
    })

results_df = pd.DataFrame(results)
results_df


,Film Name,True Genre,Template 1 Prediction,Template 2 Prediction
0,ملك البترول,كوميدي,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يحتو...
1,الكل يغنى,دراما,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يركز...
2,وغدا,دراما,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يركز...
3,نداء الحب,دراما,رومانسي,- النوع الرئيسي: رومانسي\n- التبرير: الفيلم ير...
4,كابتن مصر,رياضي,رومانسي,- النوع الرئيسي: رياضي\n- التبرير: يعتبر الفيل...
5,الهارب,ﺗﺸﻮﻳﻖ ﻭﺇﺛﺎﺭﺓ,جريمة,- النوع الرئيسي: رعب\n- التبرير: الفيلم يحتوي ...
6,المنتقم,عائلي,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يحتو...
7,نساء الليل,دراما,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يركز...
8,المرأة شيطان,دراما,دراما,- النوع الرئيسي: دراما\n- التبرير: الفيلم يحتو...
9,جيتار الحب (غيتار الحب),رومانسي,رومانسي,- النوع الرئيسي: دراما\n- التبرير: يعتبر الفيل...


from matplotlib import pyplot as plt
import seaborn as sns
results_df.groupby('True Genre').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
results_df.groupby('Template 1 Prediction').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Template 1 Prediction'].value_counts()
    for x_label, grp in results_df.groupby('True Genre')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('True Genre')
_ = plt.ylabel('Template 1 Prediction')

## Template Comparison

After testing both templates on:

1. Custom user-written film descriptions in Arabic.
2. Real summaries (**ملخص**) from the CIMA dataset,

we observed the following:

### Template 1 – Direct Classification

- **Output format:**
  - Returns only one word or phrase representing the film genre.
  - Example: `دراما`, `رومانسي`, `كوميدي`.

- **Advantages:**
  - Very easy to use in the system (we can display the genre directly).
  - The output is short and clear, and does not require any additional parsing.
  - Suitable when we only need the final classification and nothing else.

- **Disadvantages:**
  - Does not explain *why* this genre was chosen.
  - Does not show if there are other possible genres (no secondary options).

### Template 2 – Analytical Explanation

- **Output format:**
  - The answer is structured and usually contains:
    - `النوع الرئيسي` (main genre)
    - `التبرير` (justification)
    - `تصنيف ثانوي محتمل` (optional secondary genre)

- **Advantages:**
  - Provides deeper understanding of why the model chose this genre.
  - Useful during development, analysis, and debugging of the system.
  - Helps the team or the end user to trust the model’s recommendation.

- **Disadvantages:**
  - The text is longer and needs extra processing to extract only the main genre.
  - Less convenient if the user interface only needs to show a single final genre.


## Final Choice of Prompt Template

For the final system we want to build, the main goal is:

> The user writes a film description in Arabic,  
> and the system returns the film genre (**تصنيف الفيلم**) directly.

Based on this goal, we selected **Template 1** as the main prompt template for the deployed system, for the following reasons:

1. **Simplicity of the output**  
   Template 1 returns only the genre (for example: دراما، رومانسي، كوميدي).  
   This fits the user interface, which needs to display one clear genre without extra text.

2. **Easy integration in the system**  
   Because the output is just one label, we do not need to parse a long explanation or split the text into parts.  
   This makes the integration with the front-end and the API layer simpler and less error-prone.

3. **Consistency with the CIMA dataset**  
   In the prompt, we restrict the model to choose from the same list of genres that already exists in the dataset (**تصنيف الفيلم**).  
   This reduces the chance of getting strange or invalid genre names that are not present in our database.

However, **Template 2** is still very useful during development and internal testing. We can use it when we need to:

- Understand why the model predicted a specific genre.
- Investigate incorrect or unclear classifications.
- Check the logical reasoning of the model for certain edge cases.

In summary, for the **final integrated system** we use **Template 1** because it produces a clean and simple genre label that is easy to display and store.  
We keep **Template 2** as a supporting tool for analysis, debugging, and improving the overall quality of the recommendations.

